In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from datetime import date
import holidays
import matplotlib.pyplot as plt
import seaborn as sns
import calendar


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_original = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test_original  = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
train=train_original.copy()
test=test_original.copy()

# check data

In [ ]:
cols_with_missing_train = [col for col in train.columns if train[col].isnull().any()]
cols_with_missing_test = [col for col in test.columns if test[col].isnull().any()]
print(cols_with_missing_train,cols_with_missing_test)

**train data and test data has no missing value**

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
print('---------train data types is here-----------')
print(train.dtypes)
print('---------train data shape is here-----------')
print(train.shape)
print('---------test data types is here-----------')
print(test.dtypes)
print('---------test data shape is here-----------')
print(test.shape)
print('---------train data nique is here-----------')
print(train.nunique())
print('---------test data nique is here-----------')
print(test.nunique())

from train and test data situation, we will predict how many number would be sold at each country ,store ,product
 in a day in 2019

In [ ]:
for df in [train, test]:
    df['date'] = pd.to_datetime(df.date)

# check time series graph

In [ ]:
plt.figure(figsize=(25,6))

sns.lineplot(x=train['date'], y=train['num_sold'])
plt.ylabel("num_sold")

train data has some fature
1. new year holiday season is most high number of sold
2. around March and April are little bit high number of sold
3. It is increasing year by year

**look Time series graph at each country**

In [ ]:
plt.figure(figsize=(25,6))
sns.lineplot(x=train['date'], y=train['num_sold'] ,hue = train['country'])
plt.ylabel("num_sold")

1. each country has almost same feature with  Time series graph
2. 3 country data have same trend at each other
3. Norway sold most,Sweden sold second but Sweden and Finland is almost same

**look Time series graph at each product**

In [ ]:
plt.figure(figsize=(25,6))
sns.lineplot(x=train['date'], y=train['num_sold'] ,hue = train['product'])
plt.ylabel("num_sold")

3 product has different trend each other
1. Kaggle Hat sold most in 3products and sold number is decrease March to October ,increase October to January every year
2. Kaggle Mug sold second in 3products and sold most in new year holiday season ,sold least at around July
3. Kaggle Sticker sold most in new year holiday season other season is almost same

**look Time series graph at each store**

In [ ]:
plt.figure(figsize=(25,6))
sns.lineplot(x=train['date'], y=train['num_sold'] ,hue = train['store'])
plt.ylabel("num_sold")

1. KaggleRama sold more than Kaggle Mart
2. Trend is almost same

In [ ]:
train[['year', 'month','day']] = (train_original["date"].str.split("-", expand=True))
train['day_of_week'] = train["date"].dt.weekday

In [ ]:
plt.figure(figsize=(15,6))
sns.barplot(x=train['month'], y=train['num_sold'])
plt.ylabel("num_sold")

December sold most,April is second, January is third

**look at relation month and num_sold with each product becouse the trend was different**

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(15,6))
sns.barplot(x=train['month'], y=train['num_sold'] ,hue = train['product'])
plt.ylabel("num_sold")

Kaggle Hat sold April ,not December. I think it is notable feature.

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(15,6))
sns.barplot(x=train['day_of_week'], y=train['num_sold'])
plt.ylabel("num_sold")

Satarday and Sunday sold most

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(15,6))
sns.barplot(x=train['day'], y=train['num_sold'])
plt.ylabel("num_sold")

almost same but day of 31 is sold most

# I will create model with this EDA